In [1]:
import import_ipynb
import os
from webdriver_manager.chrome import ChromeDriverManager
from category_list_clean import category_clean_data
from channel_detail_csv import channel_details_csv
from scrape_code import top_youtuber
from get_info_youtube import get_user_info
from get_top_youtube_info import get_top_user_info

importing Jupyter notebook from category_list_clean.ipynb
importing Jupyter notebook from channel_detail_csv.ipynb
importing Jupyter notebook from scrape_code.ipynb


In [2]:
#Calling code to retrieve the cleaned category list
category_list = category_clean_data()

category_list

,Category_id,Category_Name
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


In [3]:
#Channel details from csv file
channel_info =channel_details_csv()
channel_info

,channelid,videocategoryid,channelviewcount,videocount,subscribercount
0,UCdzU3DSGzyWzN2118yd9X9g,22,14654,30,18
1,UC0UnhAG47DRyVZGVcbhAXhQ,10,105909,51,184
2,UCXjtAvK5P3wXBGh0vbGylzg,27,48265,72,338
3,UCeKHMeUlcLNPLCLUfZUQI2w,26,2116722,172,22051
4,UCNWPDyaWf2eAHnofFLSnEMg,20,1649075,2777,220
...,...,...,...,...,...
4995,UCRJQw-oofyAgFv3JlnIAxZQ,24,359351710,1651,689911
4996,UCOwsbmIzp5OJNQjx6jOW2hg,24,13693848,120,8262
4997,UCVbIr9Qmt-r4isCQI963xgw,10,329167,9,0
4998,UCieL5ZyVlV6Sd9k2PnAN_Lw,20,12131,43,10


In [4]:
#Top channels scrapped from website using selenium & beautiful soup
user_details = top_youtuber()
user_details

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [C:\Users\jegan\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


,Channel_id,Channel_title,Category_name
0,UCq-Fj5jknLsUf-MWSy4_brA,T-Series,Music
1,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery Rhymes,Education
2,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,Gaming
3,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,Entertainment
4,UCk8GzjMOrta8yxDcKfylJYw,✿ Kids Diana Show,Film & Animation
...,...,...,...
95,UClZF7OsrECe3t_ADrjZEh_Q,Geet MP3,Music
96,UCbrd1vu4_7qIE6IPV_dA-OA,GENIAL,Howto & Style
97,UCtW7qWjpCZ8zps-Cf2NF26w,Gaane Sune Ansune,Music
98,UCLsooMJoIpl_7ux2jvdPB-Q,Super Simple Songs - Kids Songs,Education


In [ ]:
#Retreive more information on channels from csv file through youtube API
channel_detailed_info_csv = get_user_info()
channel_detailed_info_csv

In [ ]:
#Retreive more information on channels from scrapped through youtube API
top_channel_detailed_info = get_user_info()
top_channel_detailed_info